In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('data.parquet', engine='pyarrow')

# sort data raw by timestamp
df.sort_values(by='time', inplace=True)

In [ ]:
# Check the size of the dataset
print(f'Number of rows: {df.shape[0]}')
print(f'Number of columns: {df.shape[1]}')

# Check starting and ending time lag of the dataset
print(f'First time lag: {df.time.iloc[0]}')
print(f'Last time lag: {df.time.iloc[-1]}')

In [ ]:
df.tail(1)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# line plot of close value over time
df.plot(x='time', y='close', figsize=(16,8))